## Entorno

In [1]:
!pip install azureml-sdk

  Using cached azureml_sdk-1.59.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached azureml_core-1.59.0.post2-py3-none-any.whl.metadata (3.4 kB)
  Using cached azureml_dataset_runtime-1.59.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached azureml_train_core-1.59.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached azureml_train_automl_client-1.59.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached azureml_pipeline-1.59.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached backports.tempfile-1.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
  Using cached msal-1.32.0-py3-none-any.whl.metadata (11 kB)
  Using cached msal_extensions-1.3.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached knack-0.12.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached azure_core-1.33.0-py3-none-any.whl.metadata (42 kB)
  Using cached pkginfo-1.12.1.2-py3-none-any.whl.metadata (13 kB)
  Using cached argcomplete-3.6.2-py3-none-any.whl.metadata (16 kB)
 


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\pedro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Despliegue en la nube

In [ ]:
from azureml.core import Workspace

ws = Workspace.create(name="Actividad_21",
                      subscription_id = "id",
                      resource_group = "__hw21__",
                      location = "centralindia") #centralindia

Deploying AppInsights with name actividainsightsa830eb0a.
Deployed AppInsights with name actividainsightsa830eb0a. Took 6.97 seconds.
Deploying StorageAccount with name actividastoragee81b5be7b.
Deploying KeyVault with name actividakeyvault496ef99a.
Deployed KeyVault with name actividakeyvault496ef99a. Took 24.66 seconds.
Deployed StorageAccount with name actividastoragee81b5be7b. Took 36.88 seconds.
Deploying Workspace with name Actividad_21.
Deployed Workspace with name Actividad_21. Took 85.51 seconds.


In [10]:
from azureml.core.model import Model

registered_model = Model.register(model_path="model.pkl",
                                  model_name="model",
                                  workspace=ws)

Registering model model


In [11]:
from azureml.core.environment import Environment
virtual_env = Environment("env-4-housing")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=[
    'pandas',
    'scikit-learn'
    ])

In [12]:
import json

umb = open("umbral.json", "r")
umb = json.load(umb)
umbral = umb["umbral"][0]

scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('model')
  model = joblib.load(model_path)

def run(raw_data):
  try: ## Try la predicción.
    data = json.loads(raw_data)['data'][0]
    data = pd.DataFrame(data)


    result = model.predict(data_dummies).tolist()
    result_sigmoid = sigmoid(result)
    umbral = {umbral}
    result_finals = [1 if x > umbral else 0 for x in result_sigmoid]

    return json.dumps(result_finals)
  except Exception as e:
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2)

service = Model.deploy(workspace=ws,
                       name='bankrupcy',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\pedro\AppData\Local\Temp\ipykernel_16756\817183983.py:10: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [15]:
service.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"


In [16]:
scoring_uri = service.scoring_uri

In [17]:
scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()